# 데이터 파이프라인 구축을 위한 검증 테스트

1. 국내 데이터 소스

- 1-1. 지피코리아
    - RAG의 메인 지식 베이스용으로 활용
    - BeatifulSoup로 긁어오기

- 1-2. 네이버 스포츠(모터스포츠 일반)

-------------------------------------------

2. 해외 데이터 소스

- 크롤링 -> Gemini로 번역 -> DB에 저장
- 크롤링 -> DB에 저장 -> english encoder 사용

- 2-1. Autosport/Motorsport.com
    - JustText, Trafilatura << 본문 추출 라이브러리 사용

- 2-2. Formula1.com(공홈)
    - Race Strategy Analysis
    - Selenium , Playwright


-----------------------------------------------

3. FIA 공식 규정집

### Trial 1 >> GPKorea

In [ ]:
import requests
from bs4 import BeautifulSoup

def check_what_is_in_s1n2():
    # 님께서 설정하신 그 URL 그대로!
    url = "http://www.gpkorea.com/news/articleList.html?sc_section_code=S1N2&view_type=sm"
    
    headers = {
        'User-Agent': 'Mozilla/5.0'
    }
    
    res = requests.get(url, headers=headers)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.content, 'html.parser')
    
    print(f"접속 URL: {url}")
    print(f"응답 상태: {res.status_code}")
    
    # 지피코리아의 모든 가능한 제목 태그 패턴을 다 찔러봅니다.
    selectors = [
        ".list-titles a",
        ".list-block .tit a",
        "h4 a",
        ".list-body a" 
    ]
    
    found_any = False
    
    for selector in selectors:
        titles = soup.select(selector)
        if titles:
            print(f"\n[패턴 발견] '{selector}'로 {len(titles)}개를 찾았습니다!")
            print("--- 상위 5개 제목 ---")
            for i, t in enumerate(titles[:5]):
                print(f"{i+1}. {t.get_text(strip=True)}")
            found_any = True
            break # 하나 찾았으면 중단
            
    if not found_any:
        print("\n모든 패턴으로도 제목을 못 찾았습니다. HTML 구조가 완전히 다른가 봅니다.")
        # 이 경우 HTML을 통째로 봐야 알 수 있어서, 혹시나 해서 body 일부를 출력
        print("HTML 일부:", soup.body.get_text()[:200])

check_what_is_in_s1n2()

접속 URL: http://www.gpkorea.com/news/articleList.html?sc_section_code=S1N2&view_type=sm
응답 상태: 200

[패턴 발견] 'h4 a'로 21개를 찾았습니다!
--- 상위 5개 제목 ---
1. '챔프 탄생' 권오탁, 문체부장관배 KIC컵 카트레이싱 종합우승
2. WRC 최종전 '사우디 랠리' 개막…한국타이어, '다이나프로' 독점 공급
3. F1 맥라렌 노리스·피아스트리 `충격의 실격`…스키드 블록 규정 위반
4. "이럴 수가..." F1 페르스타펜, 라스베이거스GP 우승 '24점차 좁혔다'
5. F1 페르스타펜 “내년 시즌엔 3번과 69번으로 바꾸겠다"


In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def crawl_gpkorea_final():
    # 1. URL: 모터스포츠 종합 섹션 (S1N2)
    url = "http://www.gpkorea.com/news/articleList.html?sc_section_code=S1N2&view_type=sm"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    # 2. 요청 및 인코딩 설정
    res = requests.get(url, headers=headers)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.content, 'html.parser')
    
    articles = []
    
    # 3. F1 키워드 (필터망)
    # 화면에 보이는 '맥라렌', '노리스', '페르스타펜' 등이 다 포함되어 있습니다.
    f1_keywords = ['F1', '포뮬러1', '포뮬러 1', '그랑프리', 'GP', 
                   '베르스타펜', '해밀턴', '르클레르', '페라리', '레드불', '메르세데스', '맥라렌', '노리스']

    # 4. ★ 핵심 수정: 디버그에서 성공한 'h4 a' 선택자 사용
    rows = soup.select("h4 a") 

    print(f"전체 기사 {len(rows)}개 중 F1 기사를 선별합니다...")

    for row in rows:
        title = row.get_text(strip=True)
        link = "http://www.gpkorea.com" + row['href']
        
        # 5. 필터링 로직
        if any(keyword in title for keyword in f1_keywords):
            print(f"[수집 완료] {title}")
            
            try:
                # 상세 페이지 본문 수집
                sub_res = requests.get(link, headers=headers)
                sub_res.encoding = 'utf-8'
                sub_soup = BeautifulSoup(sub_res.content, 'html.parser')
                
                # 본문 ID
                content_div = sub_soup.select_one("#article-view-content-div")
                
                if content_div:
                    content = content_div.get_text(strip=True)
                    
                    # 노이즈 제거
                    if "Copyright" in content:
                        content = content.split("Copyright")[0]
                    
                    articles.append({
                        "title": title,
                        "link": link,
                        "context": content
                    })
                    time.sleep(0.1) # 매너 딜레이
                
            except Exception as e:
                print(f"  └ 에러: {e}")
                continue
        else:
            # F1 기사가 아니면 패스 (로그 확인용)
            # print(f"  [패스] {title}") 
            pass
            
    return pd.DataFrame(articles)

# 실행
df = crawl_gpkorea_final()
print(f"최종 수집 결과: {len(df)}개의 F1 기사 확보!")
if not df.empty:
    print(df[['title']].head())

전체 기사 21개 중 F1 기사를 선별합니다...
[수집 완료] F1 맥라렌 노리스·피아스트리 `충격의 실격`…스키드 블록 규정 위반
[수집 완료] "이럴 수가..." F1 페르스타펜, 라스베이거스GP 우승 '24점차 좁혔다'
[수집 완료] F1 페르스타펜 “내년 시즌엔 3번과 69번으로 바꾸겠다"
[수집 완료] `페라리 F1 영입설' 호너...윌리엄스-캐딜락-애스턴마틴은 "NO!"
[수집 완료] “F1에서 배우는 조직 전략”…변동식 전 협회장 `F1 리더십` 출간
[수집 완료] "F1이 별거야" 전기 콘셉트카 'DGR-Lola' 공개…모나코 서킷 11초 앞서
[수집 완료] '데뷔 초읽기' 아우디 F1, 최강 드라이버 페르스타펜 영입 노리나
[수집 완료] 페라리 F1팀, 기업가치 MLB 제치고 9조 원대...'미국의 힘'
[수집 완료] '티타늄-블랙-레드' 아우디, F1 데뷔 앞두고 `R26 콘셉트` 공개…“2030년 챔피언 도전”
최종 수집 결과: 9개의 F1 기사 확보!
                                         title
0       F1 맥라렌 노리스·피아스트리 `충격의 실격`…스키드 블록 규정 위반
1  "이럴 수가..." F1 페르스타펜, 라스베이거스GP 우승 '24점차 좁혔다'
2             F1 페르스타펜 “내년 시즌엔 3번과 69번으로 바꾸겠다"
3      `페라리 F1 영입설' 호너...윌리엄스-캐딜락-애스턴마틴은 "NO!"
4       “F1에서 배우는 조직 전략”…변동식 전 협회장 `F1 리더십` 출간


In [20]:
df.head()

,title,link,context
0,F1 맥라렌 노리스·피아스트리 `충격의 실격`…스키드 블록 규정 위반,http://www.gpkorea.com/news/articleView.html?i...,사진=맥라렌 F1 팀 페이스북포뮬러 원(F1) 맥라렌 팀이 라스베이거스 그랑프리 결...
1,"""이럴 수가..."" F1 페르스타펜, 라스베이거스GP 우승 '24점차 좁혔다'",http://www.gpkorea.com/news/articleView.html?i...,사진=레드불 F1 레이싱 페이스북F1 미국 라스베이거스 대회에서 맥스 페르스타펜(레...
2,"F1 페르스타펜 “내년 시즌엔 3번과 69번으로 바꾸겠다""",http://www.gpkorea.com/news/articleView.html?i...,사진=레드불 레이싱 페이스북맥스 페르스타펜이 내년 시즌 자신의 경주차 넘버로 3과 ...
3,"`페라리 F1 영입설' 호너...윌리엄스-캐딜락-애스턴마틴은 ""NO!""",http://www.gpkorea.com/news/articleView.html?i...,사진=레드불 레이싱크리스천 호너(51)가 페라리 F1에 전격 영입된다는 소문이 돌고...
4,“F1에서 배우는 조직 전략”…변동식 전 협회장 `F1 리더십` 출간,http://www.gpkorea.com/news/articleView.html?i...,변동식 전 대한자동차경주협회장이 신간 ‘F1 리더십(메디치미디어 펴냄)’을 선보여 ...


### Trial 2 >> Autosport



In [18]:
# Request 로 1차 시도

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

def crawl_autosport_f1():
    url = "https://www.autosport.com/f1/news/"
    
    # 봇 탐지를 피하기 위한 리얼한 헤더 설정
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Referer': 'https://www.google.com/'
    }
    
    print(f"Autosport 잠입 시도: {url}")
    
    try:
        # 타임아웃 10초 설정
        res = requests.get(url, headers=headers, timeout=10)
        print(f"응답 코드: {res.status_code}") # 200이면 성공!
        
        if res.status_code == 403:
            print("403 Forbidden: 봇 탐지됨! (Selenium 필요)")
            return pd.DataFrame()
            
    except Exception as e:
        print(f"접속 에러: {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(res.content, 'html.parser')
    articles = []
    
    # Autosport 구조상 모든 링크(a)를 긁어서 뉴스 링크만 필터링하는 게 가장 확실함
    # (클래스명이 자주 바뀌기 때문)
    links = soup.select("a")
    
    print(f"링크 {len(links)}개 분석 중...")
    
    unique_links = set() # 중복 방지용
    
    for row in links:
        title = row.get_text(strip=True)
        href = row.get('href', '')
        
        # 필터링 조건:
        # 1. 링크에 '/f1/news/'가 포함되어야 함
        # 2. 제목 길이가 적당히 길어야 함 (메뉴 버튼 제외)
        # 3. 중복된 링크가 아니어야 함
        if '/f1/news/' in href and len(title) > 15:
            full_link = "https://www.autosport.com" + href if not href.startswith('http') else href
            
            if full_link not in unique_links:
                unique_links.add(full_link)
                
                print(f"✅ [발견] {title[:30]}...") # 제목이 기니까 앞부분만
                
                articles.append({
                    "title": title,
                    "link": full_link,
                    "source": "Autosport"
                })
                
                if len(articles) >= 10: # 테스트니까 10개만
                    break
    
    return pd.DataFrame(articles)

# 실행
df_autosport = crawl_autosport_f1()

if not df_autosport.empty:
    print(f"성공! {len(df_autosport)}개의 영문 기사를 확보했습니다.")
    print(df_autosport.head())
else:
    print("\n아무것도 못 건졌거나 차단당했습니다.")

Autosport 잠입 시도: https://www.autosport.com/f1/news/
응답 코드: 200
링크 568개 분석 중...
✅ [발견] Pirelli's C6 experiment didn't...
✅ [발견] Why Las Vegas was typical of F...
✅ [발견] Why there's no reason to panic...
✅ [발견] Tsunoda moans "everything is g...
✅ [발견] Should McLaren back Norris now...
✅ [발견] Pirelli abandons C6 tyre as it...
✅ [발견] Antonelli "didn't really feel"...
✅ [발견] Ferrari must "calm down" after...
✅ [발견] McLaren telling Norris to atta...
✅ [발견] Why McLaren was the surprise c...
성공! 10개의 영문 기사를 확보했습니다.
                                               title  \
0  Pirelli's C6 experiment didn't work - but nor ...   
1  Why Las Vegas was typical of Ferrari’s sub-opt...   
2  Why there's no reason to panic for Norris and ...   
3  Tsunoda moans "everything is going against me"...   
4  Should McLaren back Norris now in F1 title fig...   

                                                link     source  
0  https://www.autosport.com/f1/news/pirellis-c6-...  Autosport  
1  https://www.aut

In [19]:
df_autosport.head()

,title,link,source
0,Pirelli's C6 experiment didn't work - but nor ...,https://www.autosport.com/f1/news/pirellis-c6-...,Autosport
1,Why Las Vegas was typical of Ferrari’s sub-opt...,https://www.autosport.com/f1/news/las-vegas-ty...,Autosport
2,Why there's no reason to panic for Norris and ...,https://www.autosport.com/f1/news/why-theres-n...,Autosport
3,"Tsunoda moans ""everything is going against me""...",https://www.autosport.com/f1/news/everything-i...,Autosport
4,Should McLaren back Norris now in F1 title fig...,https://www.autosport.com/f1/news/should-mclar...,Autosport


In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

# --- 1. 공통 헤더 (스텔스 모드) ---
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Referer': 'https://www.google.com/'
}

def extract_autosport_content(url):
    """
    [수정버전] 클래스 이름 의존성을 제거하고, 
    모든 p태그를 긁어서 기사 같은 것만 남기는 방식
    """
    try:
        # 매너 딜레이
        time.sleep(random.uniform(1.0, 2.0))
        
        res = requests.get(url, headers=HEADERS, timeout=10)
        if res.status_code != 200:
            print(f"  └ 접속 실패 (Code: {res.status_code})")
            return ""
            
        soup = BeautifulSoup(res.content, 'html.parser')
        
        # 1. 봇 차단 여부 확인 (혹시 캡차가 떴는지 확인)
        page_title = soup.title.get_text(strip=True) if soup.title else "No Title"
        if "Bot" in page_title or "Access Denied" in page_title:
             print(f"  └ 🚨 봇 탐지됨! (Page Title: {page_title})")
             return ""

        # 2. [전략 변경] 특정 Div 찾지 말고, 그냥 모든 p 태그 가져오기
        # Autosport 기사는 보통 article 태그나 main 태그 안에 있습니다.
        # 범위를 좁히기 위해 article 태그 먼저 시도, 없으면 전체에서 찾기
        container = soup.find('article')
        if not container:
            container = soup # 없으면 전체 HTML에서 찾음
            
        paragraphs = container.find_all('p')
        
        clean_text = []
        for p in paragraphs:
            text = p.get_text(strip=True)
            
            # 3. [노이즈 필터링]
            # - 너무 짧은 문장 (메뉴, 링크 등) 제외 (50자 이상만)
            # - 'Read Also', 'Photo by' 같은 거 제외
            if len(text) > 50 and "Read Also:" not in text and "Photo by" not in text:
                clean_text.append(text)
        
        # 수집된 문장이 3개 미만이면 실패로 간주 (너무 적음)
        if len(clean_text) < 3:
            print(f"  └ 텍스트 추출 실패 (유효 문장 부족, HTML 구조 확인 필요)")
            # 디버깅용: 도대체 뭘 가져왔는지 확인
            # print(f"    (Debug: Page Title - {page_title})") 
            return ""

        return " ".join(clean_text)

    except Exception as e:
        print(f"  └ 에러 발생: {e}")
        return ""

def crawl_autosport_full():
    """
    목록 수집 + 본문 수집을 합친 메인 함수
    """
    base_url = "https://www.autosport.com/f1/news/"
    print(f"Autosport 접근 중: {base_url}")
    
    res = requests.get(base_url, headers=HEADERS)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    links = soup.select("a")
    articles = []
    unique_links = set()
    
    print("기사 리스트 분석 및 본문 수집 시작...")
    
    count = 0
    for row in links:
        title = row.get_text(strip=True)
        href = row.get('href', '')
        
        # 필터링: F1 뉴스 링크이고 제목이 좀 긴 것
        if '/f1/news/' in href and len(title) > 20:
            full_link = "https://www.autosport.com" + href if not href.startswith('http') else href
            
            if full_link not in unique_links:
                unique_links.add(full_link)
                print(f"[Target] {title[:40]}...")
                
                # ★ 여기서 본문 추출 함수 호출!
                content = extract_autosport_content(full_link)
                
                if content:
                    articles.append({
                        "title": title,
                        "link": full_link,
                        "context": content, # 이게 우리가 원하던 것!
                        "source": "Autosport"
                    })
                    print(f"  └ 본문 수집 완료 ({len(content)}자)")
                    count += 1
                else:
                    print("  └ 본문 수집 실패 (Skipped)")
                
                if count >= 5: # 테스트니까 5개만 합시다
                    break
    
    return pd.DataFrame(articles)

# --- 실행 ---
df_autosport_full = crawl_autosport_full()

if not df_autosport_full.empty:
    print(f"\n성공! 총 {len(df_autosport_full)}개의 기사 본문까지 확보했습니다.")
    # 첫 번째 기사의 본문 앞부분만 살짝 맛보기
    print("\n[첫 번째 기사 본문 미리보기]")
    print(df_autosport_full.iloc[0]['context'][:300] + "...")
else:
    print("\n수집 실패. 헤더나 선택자를 다시 확인해야 합니다.")

Autosport 접근 중: https://www.autosport.com/f1/news/
기사 리스트 분석 및 본문 수집 시작...
[Target] Pirelli's C6 experiment didn't work - bu...
  └ 본문 수집 완료 (2996자)
[Target] Why Las Vegas was typical of Ferrari’s s...
  └ 본문 수집 완료 (6502자)
[Target] Why there's no reason to panic for Norri...
  └ 본문 수집 완료 (7526자)
[Target] Tsunoda moans "everything is going again...
  └ 본문 수집 완료 (4437자)
[Target] Should McLaren back Norris now in F1 tit...
  └ 본문 수집 완료 (9492자)

성공! 총 5개의 기사 본문까지 확보했습니다.

[첫 번째 기사 본문 미리보기]
Manage alerts on breaking news and favorite drivers Razgatlioglu aims to challenge Quartararo in rookie MotoGP season Pirelli's C6 experiment didn't work - but nor have the other 2025 F1 tyre trials Why Las Vegas was typical of Ferrari’s sub-optimal 2025 F1 season How Toyota is revamping its GR010 L...


### Trial 3

- F1 공홈 >> Race Strategy analysis

In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import random

In [ ]:
def crawl_f1_analysis_only():
    chrome_options = Options()
    # ★ WSL 환경 필수 옵션 (헤드리스)
    chrome_options.add_argument("--headless") 
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("window-size=1920,1080")
    
    # 봇 탐지 회피 헤더
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

    print("중장비(Selenium) 가동: Analysis 섹션 공략...")
    
    # 드라이버 실행
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    # ★ 타겟 변경: Analysis 태그 페이지 (여기가 전략 리포트 맛집)
    url = "https://www.formula1.com/en/latest/tags/analysis.3HkjTN75peeCOsSegCyOWi"
    print(f"진입 URL: {url}")
    
    try:
        driver.get(url)
        time.sleep(3) # 로딩 대기
        
        # 쿠키 팝업 처리 (혹시 뜨면 닫기)
        try:
            cookie_btn = driver.find_element(By.ID, "sp-cc-accept")
            cookie_btn.click()
        except:
            pass

        # 링크 수집 시작
        links = driver.find_elements(By.TAG_NAME, "a")
        print(f"전체 링크 {len(links)}개 1차 스캔 중...")
        
        target_links = []
        seen_urls = set() # 중복 URL 방지용
        
        for link in links:
            try:
                href = link.get_attribute('href')
                title = link.text.strip()
                
                # ★ 필터링 강화 (Strict Mode) ★
                # 1. 주소에 '/en/latest/article'이 있어야 함 (뉴스 기사만)
                # 2. 제목(title)이 비어있으면 안 됨 (이미지 링크, 아이콘 등 제외)
                # 3. 'Video' 나 'Podcast'는 제외 (텍스트 분석용이니까)
                if href and '/en/latest/article' in href and title:
                    
                    if "Video" not in title and "Podcast" not in title:
                        if href not in seen_urls:
                            seen_urls.add(href)
                            target_links.append({"href": href, "title": title})
                            print(f"  [Target] {title[:40]}...")
            except:
                continue
        
        print(f"총 {len(target_links)}개의 분석 리포트 후보 선정 완료.")
        
        # 본문 수집 (상위 5개만)
        articles = []
        for target in target_links[:5]:
            print(f"  본문 추출 중: {target['title'][:20]}...")
            
            try:
                driver.get(target['href'])
                time.sleep(random.uniform(1.5, 3.0))
                
                # 본문 추출 (p 태그 전체 스캔 전략)
                paragraphs = driver.find_elements(By.TAG_NAME, "p")
                
                content = []
                for p in paragraphs:
                    text = p.text.strip()
                    # 노이즈 제거: 너무 짧거나 쿠키/저작권 관련 문구 제외
                    if len(text) > 40 and "cookie" not in text.lower() and "©" not in text:
                        content.append(text)
                
                full_text = " ".join(content)
                
                if len(full_text) > 500: # 내용이 충분한 경우만 저장
                    articles.append({
                        "title": target['title'],
                        "link": target['href'],
                        "context": full_text,
                        "source": "F1 Official Analysis"
                    })
                    print(f"    성공 ({len(full_text)}자)")
                else:
                    print("    실패 (내용 부족)")

            except Exception as e:
                print(f"    에러: {e}")

    except Exception as e:
        print(f"치명적 에러: {e}")
        
    finally:
        driver.quit()
        
    return pd.DataFrame(articles)

# 실행
df_analysis = crawl_f1_analysis_only()

if not df_analysis.empty:
    print(f"\n최종 결과: {len(df_analysis)}개의 고품질 전략 리포트 확보!")
    print(df_analysis[['title']].head())

중장비(Selenium) 가동: Analysis 섹션 공략...
🌍 진입 URL: https://www.formula1.com/en/latest/tags/analysis.3HkjTN75peeCOsSegCyOWi
전체 링크 150개 1차 스캔 중...
  [Target] What are the tactical options for the La...
  [Target] Did Piastri deserve his penalty in Sao P...
  [Target] What are the tactical options for the Sa...
  [Target] What are the tactical options for the Me...
  [Target] What are the tactical options for the Un...
  [Target] Are Russell and Antonelli Mercedes' long...
  [Target] What are the tactical options for the Si...
  [Target] What are the tactical options for the Az...
  [Target] What are the tactical options for the It...
  [Target] Why Alpine were keen to lock down Gasly ...
  [Target] Why Cadillac decided to go for Perez and...
  [Target] What are the tactical options for the Hu...
  [Target] What are the tactical options for the Be...
  [Target] The key questions as Horner leaves Red B...
  [Target] What are the tactical options for the Br...
  [Target] What are the tactical op

In [33]:
df_analysis.head()

,title,link,context,source
0,What are the tactical options for the Las Vega...,https://www.formula1.com/en/latest/article/str...,Matt Youson takes a look at the different pit ...,F1 Official Analysis
1,Did Piastri deserve his penalty in Sao Paulo?,https://www.formula1.com/en/latest/article/pal...,Former F1 driver Jolyon Palmer breaks down the...,F1 Official Analysis
2,What are the tactical options for the Sao Paul...,https://www.formula1.com/en/latest/article/str...,Matt Youson takes a look at the different pit ...,F1 Official Analysis
3,What are the tactical options for the Mexico C...,https://www.formula1.com/en/latest/article/str...,Matt Youson takes a look at the different pit ...,F1 Official Analysis
4,What are the tactical options for the United S...,https://www.formula1.com/en/latest/article/str...,Matt Youson takes a look at the different pit ...,F1 Official Analysis
